In [1]:
from yahoo_fin.stock_info import *
from yahoo_fin.options import *
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import random
import math

class OptionsData:
    
    """       
    The OPD class is used to get a DataFrame with weekly options pricing data

    Attributes
    ----------    
    ticker_list : Optional[list[str]]
        A list of strings providing the tickers to retrieve end of day trading data
        
    option_type : Optional(str)
        A string defining either calls or puts
        
    Examples
    --------    
    # By default we get TSLA & AAPL calls
    o = OptionsData()
    o.df.loc[o.df['ticker'] == 'TSLA']

    # Get RKT puts
    tickers = ['RKT']
    o = OptionsData(option_type='puts', list_of_tickers=tickers)
    rkt_puts_df = o.df    
    """
    
    def __init__(self, option_type='calls', list_of_tickers=['AAPL', 'TSLA']):

        self.option_type = option_type        
        self.list_of_tickers = list_of_tickers            
        self.df = pd.DataFrame([])
        self.get_options_data(self)        
    
    @staticmethod
    def get_options_data(self):

        # Building the analysis around the probability of trading weekly contracts
        expiration = get_expiration_dates(self.list_of_tickers[0])[0]

        options_df = pd.DataFrame()
        
        for ticker in self.list_of_tickers:
            try:
                price = get_live_price(ticker)
                if self.option_type == 'calls':
                    option = get_calls(ticker, expiration)
                elif self.option_type == 'puts':
                    option = get_puts(ticker, expiration)                    
                option['current_price'] = price
                option['ticker'] = ticker
                option['date_exp'] = expiration
                options_df = options_df.append(option)
            except:            
                print('Option contract not available on ' + str(expiration))
                pass

        # Rename the columns to follow PEP8
        options_df = options_df.rename(columns={'Contract Name': 'contract_name', 'Strike': 'strike', 'Bid': 'bid',
                                                'Ask': 'ask', 'Volume': 'vol', 'Open Interest': 'open_interest'})
        # Filter down the columns
        options_df = options_df[['contract_name', 'strike', 'bid', 'ask', 'current_price',
                                 'vol', 'open_interest', 'ticker', 'date_exp']]

        options_df['avg_price'] = (options_df['bid'] + options_df['ask']) / 2

        # Creating columns for total option cost and break even percentage
        options_df['option_cost'] = options_df['avg_price'] * 100
        options_df['be_percent'] = ((options_df['strike'] + options_df['avg_price']) / options_df['current_price']) - 1
        options_df['be_percent'] = options_df['be_percent'] * 100
        options_df['be_percent'] = options_df['be_percent'].round(2)
        options_df['be_price'] = options_df['strike'] + options_df['avg_price']

        options_df = options_df.groupby(['ticker']).apply(lambda x: self.get_mid_values(x)).reset_index(drop=True)
        self.df = options_df        
        
    @staticmethod
    def get_mid_values(x):

        """ This functions returns a new DF with the 5 closest values below & above the strike for each ticker"""

        lower_bound = x[x['strike'].round() >= x['current_price'].round()]
        upper_bound = x[x['strike'].round() < x['current_price'].round()]

        x = pd.concat([lower_bound[:5], upper_bound[-5:]])
        x = x.sort_values('strike')

        return x



In [2]:
tickers = ['RKT']
o = OptionsData(list_of_tickers=tickers)
rkt_calls_df = o.df

In [3]:
tickers = ['RKT']
o = OptionsData(option_type='puts', list_of_tickers=tickers)
rkt_puts_df = o.df

In [4]:
rkt_puts_df

,contract_name,strike,bid,ask,current_price,vol,open_interest,ticker,date_exp,avg_price,option_cost,be_percent,be_price
0,RKT201127P00018500,18.5,0.00,0.02,20.5895,28,1265,RKT,"November 27, 2020",0.010,1.0,-10.10,18.510
1,RKT201127P00019000,19.0,0.00,0.01,20.5895,274,2025,RKT,"November 27, 2020",0.005,0.5,-7.70,19.005
2,RKT201127P00019500,19.5,0.02,0.03,20.5895,218,1538,RKT,"November 27, 2020",0.025,2.5,-5.17,19.525
3,RKT201127P00020000,20.0,0.05,0.06,20.5895,1270,2872,RKT,"November 27, 2020",0.055,5.5,-2.60,20.055
4,RKT201127P00020500,20.5,0.20,0.21,20.5895,1612,2494,RKT,"November 27, 2020",0.205,20.5,0.56,20.705
5,RKT201127P00021000,21.0,0.52,0.55,20.5895,724,4788,RKT,"November 27, 2020",0.535,53.5,4.59,21.535
6,RKT201127P00021500,21.5,0.96,1.01,20.5895,64,507,RKT,"November 27, 2020",0.985,98.5,9.21,22.485
7,RKT201127P00022000,22.0,1.45,1.67,20.5895,112,659,RKT,"November 27, 2020",1.560,156.0,14.43,23.560
8,RKT201127P00022500,22.5,1.94,2.00,20.5895,69,260,RKT,"November 27, 2020",1.970,197.0,18.85,24.470
9,RKT201127P00023000,23.0,2.43,2.49,20.5895,3,123,RKT,"November 27, 2020",2.460,246.0,23.66,25.460


In [5]:
rkt_calls_df

,contract_name,strike,bid,ask,current_price,vol,open_interest,ticker,date_exp,avg_price,option_cost,be_percent,be_price
0,RKT201127C00018500,18.5,2.01,2.08,20.584999,4,406,RKT,"November 27, 2020",2.045,204.5,-0.19,20.545
1,RKT201127C00019000,19.0,1.54,1.60,20.584999,3,93,RKT,"November 27, 2020",1.570,157.0,-0.07,20.570
2,RKT201127C00019500,19.5,0.91,1.09,20.584999,6,300,RKT,"November 27, 2020",1.000,100.0,-0.41,20.500
3,RKT201127C00020000,20.0,0.58,0.63,20.584999,389,1199,RKT,"November 27, 2020",0.605,60.5,0.10,20.605
4,RKT201127C00020500,20.5,0.23,0.25,20.584999,817,911,RKT,"November 27, 2020",0.240,24.0,0.75,20.740
5,RKT201127C00021000,21.0,0.07,0.08,20.584999,5325,4322,RKT,"November 27, 2020",0.075,7.5,2.38,21.075
6,RKT201127C00021500,21.5,0.03,0.04,20.584999,2712,4262,RKT,"November 27, 2020",0.035,3.5,4.62,21.535
7,RKT201127C00022000,22.0,0.02,0.03,20.584999,2760,4504,RKT,"November 27, 2020",0.025,2.5,7.00,22.025
8,RKT201127C00022500,22.5,0.01,0.02,20.584999,1003,3320,RKT,"November 27, 2020",0.015,1.5,9.38,22.515
9,RKT201127C00023000,23.0,0.01,0.02,20.584999,599,4643,RKT,"November 27, 2020",0.015,1.5,11.80,23.015


In [8]:
o = OptionsData()
o.df.loc[o.df['ticker'] == 'TSLA']

,contract_name,strike,bid,ask,current_price,vol,open_interest,ticker,date_exp,avg_price,option_cost,be_percent,be_price
10,TSLA201127C00550000,550.0,25.05,25.40,572.219971,38959,11069,TSLA,"November 27, 2020",25.225,2522.5,0.53,575.225
11,TSLA201127C00555000,555.0,20.80,21.10,572.219971,25959,5648,TSLA,"November 27, 2020",20.950,2095.0,0.65,575.950
12,TSLA201127C00560000,560.0,17.00,17.20,572.219971,48089,9033,TSLA,"November 27, 2020",17.100,1710.0,0.85,577.100
13,TSLA201127C00565000,565.0,13.50,13.80,572.219971,38462,2821,TSLA,"November 27, 2020",13.650,1365.0,1.12,578.650
14,TSLA201127C00570000,570.0,10.20,10.45,572.219971,54829,5477,TSLA,"November 27, 2020",10.325,1032.5,1.42,580.325
15,TSLA201127C00575000,575.0,8.00,8.20,572.219971,31253,7561,TSLA,"November 27, 2020",8.100,810.0,1.90,583.100
16,TSLA201127C00580000,580.0,6.45,6.60,572.219971,40154,6297,TSLA,"November 27, 2020",6.525,652.5,2.50,586.525
17,TSLA201127C00585000,585.0,5.15,5.25,572.219971,11913,2380,TSLA,"November 27, 2020",5.200,520.0,3.14,590.200
18,TSLA201127C00590000,590.0,3.80,4.20,572.219971,13684,6262,TSLA,"November 27, 2020",4.000,400.0,3.81,594.000
19,TSLA201127C00595000,595.0,3.20,3.30,572.219971,9801,2338,TSLA,"November 27, 2020",3.250,325.0,4.55,598.250
